In [223]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn import svm
import xgboost as xgb

from codes.data import data_word2num

In [224]:
os.chdir('../car/')

In [225]:
os.listdir()

['data', 'code', '.gitignore', 'notebook', 'codes']

In [226]:
train_labels = glob.glob('data/train_labels.csv')
df_train_labels = pd.read_csv(train_labels[0])
df_train_labels

,车号,Label,CollectTime
0,1,1,2020-08-30 21:36:09
1,2,0,NaN
2,3,1,2020-08-12 08:36:46
3,4,0,NaN
4,5,1,2021-01-06 16:24:21
...,...,...,...
113,116,0,NaN
114,117,0,NaN
115,118,0,NaN
116,119,1,2021-01-07 12:46:23


In [227]:
train_labels = df_train_labels['Label'].tolist()
# train_labels

In [228]:
files = glob.glob('data/train/*.csv')
files.sort()
# files

In [231]:
data_list = []
positive_index = []

for f in files[:1]:
    df = pd.read_csv(f)
    # df = df.drop(['采集时间'], axis=1)
    df = df.dropna()
    data_list.append(np.array(df.values.tolist()))
df

,车号,采集时间,加速踏板位置,电池包主负继电器状态,电池包主正继电器状态,制动踏板状态,驾驶员离开提示,主驾驶座占用状态,驾驶员安全带状态,驾驶员需求扭矩值,手刹状态,整车钥匙状态,低压蓄电池电压,整车当前档位状态,整车当前总电流,整车当前总电压,车辆行驶里程,车速,方向盘转角
2,1,2020-08-28 06:33:14,0.0,断开,断开,未踩,No Warning,空置,未系,0.0,手刹拉起,OFF,12.43,空档,0.0,123.2,7662.0,0.0,0.0
5,1,2020-08-28 10:43:40,0.0,断开,断开,未踩,No Warning,空置,未系,0.0,手刹拉起,OFF,12.43,空档,0.0,123.2,7662.0,0.0,0.0
6,1,2020-08-28 10:33:32,0.0,断开,断开,未踩,No Warning,空置,未系,0.0,手刹拉起,OFF,12.48,空档,0.0,123.2,7662.0,0.0,0.0
7,1,2020-08-28 10:33:40,0.0,断开,断开,未踩,No Warning,空置,未系,0.0,手刹拉起,OFF,12.43,空档,0.0,123.2,7662.0,0.0,0.0
8,1,2020-08-28 10:33:44,0.0,断开,断开,未踩,No Warning,空置,未系,0.0,手刹拉起,OFF,12.43,空档,0.0,123.2,7662.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25452,1,2020-08-31 16:59:51,12.0,断开,断开,未踩,No Warning,空置,未系,0.0,手刹拉起,OFF,1.17,空档,0.0,0.0,7682.0,0.0,0.0
25458,1,2020-08-31 17:00:34,12.0,断开,断开,未踩,No Warning,空置,未系,0.0,手刹拉起,OFF,1.17,空档,0.0,0.0,7682.0,0.0,0.0
25464,1,2020-08-31 17:01:07,13.0,断开,断开,未踩,No Warning,空置,未系,0.0,手刹拉起,OFF,1.13,空档,0.0,0.0,7682.0,0.0,0.0
25465,1,2020-08-31 17:01:26,15.0,断开,断开,未踩,No Warning,空置,未系,0.0,手刹拉起,OFF,0.82,空档,0.0,120.7,7682.0,0.0,0.0


In [236]:
car_no = df['车号'].values[0]
line = df_train_labels[df_train_labels['车号'] == car_no]

print(df.index)

if line['Label'].values[0] == 1:
    collect_time = line['CollectTime'].values[0]
    index = df[df['采集时间'] == collect_time].index.values[0]

index

Int64Index([    2,     5,     6,     7,     8,     9,    10,    11,    12,
               13,
            ...
            25433, 25434, 25442, 25443, 25444, 25452, 25458, 25464, 25465,
            25467],
           dtype='int64', length=15816)


11439

In [ ]:
data_list = list(map(lambda data:list(map(lambda x:data_word2num(x), data)),data_list))

In [ ]:
data_list = list(map(lambda x:np.array(x), data_list))
data_list = np.array(data_list)

data_list.shape
data_list[0].shape

In [ ]:
clf = svm.SVC()
clf.fit(data_list, train_labels[:len(data_list)])